In [ ]:
import pandas as pd
import numpy as np
import requests
import bs4
import os
from google.colab import drive
drive.mount('/content/drive')

# 六大生活機能

## 捷運

In [ ]:
# 捷運出入口座標
df = pd.read_csv("https://data.taipei/api/getDatasetInfo/downloadResource?id=cfa4778c-62c1-497b-b704-756231de348b&rid=42694bb8-fc9d-4d82-af7a-0ad6f4c9ad57",encoding='big5')
df.head()

## 公園

### 新北市

In [ ]:
park_newtaipei = pd.read_csv("https://data.ntpc.gov.tw/api/datasets/5FE3A136-29CC-4695-A17E-6636A32C3342/csv/file",encoding='utf-8')
park_newtaipei.head()

In [ ]:
ntpc=pd.DataFrame(ntpc,columns=["pm_lon","pm_lat"])
ntpc

In [ ]:
#發現共有313個位置找不到，他們又共通點 => 經緯度皆為(120.979772	23.975744)
ntpc.astype(float)
#ntpc['pm_lon'].value_counts
count = 0
for i in range(536):
    if (ntpc.iloc[i,0] == '120.979772'):
        count += 1
        print(park_newtaipei.iloc[i,0])
print("count=",count) 

In [ ]:
#改善找不到經緯度的問題
newtaipei_position = pd.read_excel("park_newtaipei.xlsx")
newtaipei_position

In [ ]:
#620(台北)+536(新北)=1156(total)
park = pd.concat([park_taipei , newtaipei_position],axis=0)
park.to_csv("park.csv")

### 台北市

In [ ]:
park_taipei = pd.read_json("https://parks.taipei/parks/api/",encoding='big5')
park_taipei.head()

In [ ]:
# park_taipei["pm_name"]刪除裡面有綠地結尾
park_taipei["pm_name"].values

In [ ]:
p=park_taipei["pm_name"].str[-2:]=="綠地"
p

In [ ]:
for i in range(park_taipei.shape[0]):
    if (p[i]==1):
        park_taipei=park_taipei.drop([i])
park_taipei

In [ ]:
park_taipei = park_taipei[["pm_name","pm_lon","pm_lat"]]
park_taipei

## 學區

### 台北市

In [ ]:
# 臺北市110學年度國民中學學區一覽表 
df = pd.read_csv("https://data.taipei/api/getDatasetInfo/downloadResource?id=678c5215-f14a-47e3-92bd-da43f9d7c7a9&rid=a97b9fae-c20f-4a33-a5d3-fa36e64c304f", encoding='MS950')
df.head()

In [ ]:
li_df = df['學區範圍/里別'].str.replace(r".\.","").str.replace("。","").str.replace("：.*、.*共同學區","").str.replace("、"," ").str.extractall(r'\.?(.*)：?').dropna()
li_df

In [ ]:
# 改col名稱為'里'
li_df.rename(columns={0:'里', '里':0}, inplace=True)
li_df

In [ ]:
li_drop = li_df.reset_index(level=[1], drop=True) 
# li_drop = li_df.reset_index(level='match', drop=True)

# 新增欄位'國中學區'
li_drop["國中學區"]=df['學校名稱']
li_drop["區別"]=df["區別"]
li_drop

In [ ]:
li_drop['里'].str.split()

In [ ]:
li_drop['index_col']=li_drop.index
li_drop.tail(20)

In [ ]:
li_ndf = li_drop[['區別','國中學區','index_col']]
li_ndf

In [ ]:
li_drop['里'].str.split(' ',expand=True).stack().str.split(")",expand=True).stack()

In [ ]:
gs3 = li_drop.set_index('index_col')['里'] \
      .str.split(' ',expand=True) \
      .stack() \
      .str.split('）',expand=True) \
      .stack() \
      .rename('里') \
      .reset_index(level=1, drop=True) \
      .reset_index()[['里', 'index_col']] \
      .query("里!=''")

gs3

In [ ]:
map_table=pd.merge(gs3,li_ndf,on='index_col',how='right')
map_table=map_table.drop_duplicates(subset='里').reset_index(drop=True)
map_table.tail(20)

In [ ]:
# 新增鄰
for i in range(0,len(map_table.index)):  
  map_table.loc[i,'鄰'] = (map_table.loc[i,'里'] if map_table['里'].str.contains('\d',regex=True)[i] else np.nan)
map_table.tail(20)

### 新北市

In [ ]:
# 新北市109學年度公立國中學區一覽表
df = pd.read_csv("https://data.ntpc.gov.tw/api/datasets/94E8AE23-4507-4626-A77E-3A7A5DDF1013/csv/file", encoding='utf8')
df.head()


In [ ]:
ntpsjs_list=['永和國中', '海山高中國中部', '中山國中']

In [ ]:
df['sname'] = df['sname'].str.replace('海山高中\(國中部\)','海山高中國中部')
df['sname'].head(20)

In [ ]:
ntp_list=[]
for i in ntpsjs_list:
  filter = df['sname'].str.contains(i)
  filter = filter.fillna(False)
  # filter
  print(i)
  display(df[filter])
  ntp_list.append(df[filter])
  # display(ntp_df)


In [ ]:
ntp_df=pd.concat(ntp_list)
ntp_df

In [ ]:
ntp_df['fzoon'].str.replace(r'[一二三四五六]、',"")

In [ ]:
ntp_df = ntp_df.drop('pzoon',axis=1).join(ntp_df['pzoon'].str.split('、',expand=True).stack().reset_index(level=1,drop=True).rename('pzoon'))

In [ ]:
ntp_df.to_csv('/content/drive/MyDrive/學區/ntp_star_li.csv',index=False)

## 醫院

In [ ]:
hospital = pd.read_excel('/content/drive/MyDrive/ML/新增欄位/醫院.xlsx')
hospital.head()

In [ ]:
# 取得醫院的經緯度
import requests
import urllib
import json
import time
import pandas as pd
import csv
import numpy as np

def get_latitude_longtitude(address):
    # decode url
    Output = []
    for i in range(len(address)):
        print(i,address[i])
        temp = urllib.parse.quote(address[i])
        #記得改key!!!
        url = "https://maps.googleapis.com/maps/api/geocode/json?address=" + temp + "&key="
        while True:
            res = requests.get(url)
            js = json.loads(res.text)
    
            if js["status"] != "OVER_QUERY_LIMIT":
                #time.sleep(3)
                break
        #time.sleep(3)
        #若status不ok，則print[-1,-1,-1]
        if js["status"] != "OK":
            Output.append([-1,-1,-1,-1])
        else:
            #result: 得到經緯度
            #result_li: 得到li
            result = js["results"][0]["geometry"]["location"]
            result_li = js["results"][0]["address_components"]
            flag = 0
            for item in result_li:
                if item["types"][0] == "administrative_area_level_4":
                    Li = item["long_name"]
                    flag = 1
            if flag == 0 :
                Li = "No li"
            #Output = (float(result["lat"]),float(result["lng"]))
            Output.append([float(result["lat"]),float(result["lng"]),Li,address[i]])
        
    return Output


df = pd.read_excel('/content/drive/MyDrive/ML/新增欄位/醫院.xlsx')
address = df["地址"]
input_add = []
for i in range(len(address)):
    input_add.append(address[i])
print(input_add)
result = pd.DataFrame(get_latitude_longtitude(address))
print(result)

result.to_csv("/content/drive/MyDrive/ML/新增欄位/hospital.csv")

## 診所

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/ML/新增欄位/診所.xlsx')

## 便利商店

In [ ]:
conv = pd.read_excel('/content/drive/MyDrive/ML/新增欄位/conv.xlsx')

# 取得經緯度

In [ ]:
# 範例: 取得醫院的經緯度
import requests
import urllib
import json
import time
import pandas as pd
import csv
import numpy as np

def get_latitude_longtitude(address):
    # decode url
    Output = []
    for i in range(len(address)):
        print(i,address[i])
        temp = urllib.parse.quote(address[i])
        #記得改key!!!
        url = "https://maps.googleapis.com/maps/api/geocode/json?address=" + temp + "&key= 自己的google API"
        while True:
            res = requests.get(url)
            js = json.loads(res.text)
    
            if js["status"] != "OVER_QUERY_LIMIT":
                #time.sleep(3)
                break
        #time.sleep(3)
        #若status不ok，則print[-1,-1,-1]
        if js["status"] != "OK":
            Output.append([-1,-1,-1,-1])
        else:
            #result: 得到經緯度
            #result_li: 得到li
            result = js["results"][0]["geometry"]["location"]
            result_li = js["results"][0]["address_components"]
            flag = 0
            for item in result_li:
                if item["types"][0] == "administrative_area_level_4":
                    Li = item["long_name"]
                    flag = 1
            if flag == 0 :
                Li = "No li"
            #Output = (float(result["lat"]),float(result["lng"]))
            Output.append([float(result["lat"]),float(result["lng"]),Li,address[i]])
        
    return Output


df = pd.read_excel('/content/drive/MyDrive/ML/新增欄位/醫院.xlsx')
address = df["地址"]
input_add = []
for i in range(len(address)):
    input_add.append(address[i])
print(input_add)
result = pd.DataFrame(get_latitude_longtitude(address))
print(result)

result.to_csv("/content/drive/MyDrive/ML/新增欄位/hospital.csv")

# 計算生活機能距離

In [ ]:
import requests
import time
import numpy as np
import requests
import pandas as pd
from math import *
#from selenium import webdriver
#from selenium.webdriver.support.ui import Select
#from bs4 import BeautifulSoup
#options = webdriver.ChromeOptions()
#options.add_argument("headless")

# 地址的經緯度檔案
position = pd.read_csv("/content/drive/MyDrive/ML/新增欄位/ML data_outliers deletion_IQR_hospital.csv")
position = position.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1'], axis=1)
# 新增一行: distance
position['conv distance_500m'] = 0 

# 醫院

conv = pd.read_csv("/content/drive/MyDrive/ML/新增欄位/conv.csv")


#計算地址的經緯度到捷運的距離; lat:緯度; lng:經度
#lat1,lng1: 目標地址的經緯度
#lat2,lng2: 捷運座標
def Distance(lat1,lng1,lat2,lng2):
    radlat1=radians(lat1)  
    radlat2=radians(lat2)  
    a=radlat1-radlat2  
    b=radians(lng1)-radians(lng2)  
    s=2*asin(sqrt(pow(sin(a/2),2)+cos(radlat1)*cos(radlat2)*pow(sin(b/2),2)))  
    earth_radius=6378.137  
    s=abs(s*earth_radius)
    #目標地址的經緯度 & 捷運座標 相差0.3km，則return 1
    if s<=0.3: 
        return 1
    else:
        return 0
  
#dis=0
#for i in range(position.shape[0]):
# for i in range(300000):    
i=0
dis=0 
for j in range(conv.shape[0]):
  lat1 = float(position.iloc[i,20])
  lat2 = float(conv.iloc[j,1])
  lng1 = float(position.iloc[i,21])
  lng2 = float(conv.iloc[j,2])
  # print(lat1,lat2,lng1,lng2)
  if (Distance(lat1,lng1,lat2,lng2) == 1):
    print (j, conv.iloc[j,4], lat2,lng2)
  dis = dis + Distance(lat1,lng1,lat2,lng2)  
        
position.loc[i,'conv distance_500m']=dis
print(position)

# position.to_csv("/content/drive/MyDrive/ML/新增欄位/ML data_outliers deletion_IQR_hospital_conv_test.csv")